In [127]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [128]:
# 첫번째 데이터를 불러옵니다.
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


In [146]:
# 두번째 데이터를 불러옵니다.
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [130]:
# 우선, 첫번째 데이터에서 고객이름 부분의 공백을 제거해봅시다.
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace(' ', '')
kokyaku_data.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127


In [131]:
# 원래의 등록일의 타입은 object이지만, 문자열로 바꾸어준 다음 숫자만 있는지 여부를 반환해보죠.
is_digit = kokyaku_data['등록일'].astype('str').str.isdigit()
is_digit

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 등록일, Length: 200, dtype: bool

In [132]:
# 0번째 인덱스의 값은 -와 공백이 있으니 숫자만 있다고 판단하지 않았고, 1번째 인덱스는 숫자만 있으니 True를 반환했군요.
# 우리가 바꾸어주어야하는 데이터는 숫자로만 이루어진 데이터입니다.
# to_timedelta함수는 함수내의 값을 지정한 unit형태로 바꾸는데, H면 시간, D면 일과 같이 원하는 형태로 바꾸어줍니다. (시간의 차이를 의미)
# unit을 D로 설정하면 42782의 경우 42782일이 되는거죠. 그 기준이 되는 시간이 1900년 01월 01일 이므로 이 값을 더해주면 정확한 시간이 나오는거죠.
strange_date = pd.to_timedelta(kokyaku_data[is_digit]['등록일'], unit='D') + pd.to_datetime("1900-01-01")
strange_date.head()

1    2017-02-18
3    2017-05-19
4    2018-01-29
21   2017-07-06
27   2017-06-17
Name: 등록일, dtype: datetime64[ns]

In [133]:
# 다음으로 아까 등록일 중에서 숫자로만 되어있지 않은 값들도 to_datetiem을 사용해서 고쳐주도록 합시다.
# 여기서 중요한 표현이 ~입니다. ~은 부정의 의미로 해석되며 is_digit이 아닌 kokyaku_data에서 등록일 부분을 datetime으로 변경하겠다는 의미입니다.
notstrange_date = pd.to_datetime(kokyaku_data[~is_digit]['등록일'])
notstrange_date.head()

0   2018-01-04
2   2018-01-07
5   2017-06-20
6   2018-06-11
7   2017-05-19
Name: 등록일, dtype: datetime64[ns]

In [134]:
# 이제 두 데이터를 합칠까요?
kokyaku_data['등록일'] = pd.concat([strange_date, notstrange_date])
kokyaku_data.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29


    자 그러면 여기서 궁금증을 가질 수 있습니다. concat함수는 세로로 합치는거 아닌가요? strange_date와 notstrange_date를 세로로 합치면 원래 kokyaku_data의 순서에 맞지 않은게 아닐까요?
    절반은 맞고 절반은 틀렸습니다. strange_data의 index와 notstange_data의 index를 한번 보실까요?
    strange_data는 1, 3, 4, 21, 27로 구성되어있고, notstrange_data는 0, 2, 5, 6, 7로 이루어져있네요.
    concat함수는 인덱스를 기준으로 기존에 인덱스가 존재한다면 해당 인덱스 순서대로 데이터를 세로로 결합합니다.
    이 부분 헷갈리지 않았으면 좋겠습니다.

In [135]:
# 자 그러면 kokyaku_data는 어느정도 괜찮아진 것 같군요.

In [147]:
# 이번엔 uriage_data를 봅시다.
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [148]:
# item_name이 전의 데이터셋과 비슷하게 공백처리가 되어있고, 상품A와 상품a는 같은 의미인데 이 부분을 다르다고 해석하네요.
# item_price 부분도 결측값이 보이죠? 우리가 처리해야할 부분은 이 두가지 인것 같습니다.

In [149]:
# 먼저 item_name을 고쳐줄게요.
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,NaN,김우찬
2,2019-05-11 19:42,상품a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [150]:
# 다음으로 a를 대문자화 시킬게요.
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,NaN,김우찬
2,2019-05-11 19:42,상품A,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품A,NaN,김강현


In [140]:
# 상품 A, B, C...의 가격은 모두 같겠죠?
# 우선 결측치 값을 골라낸 이후에 아이템 이름에 맞춰서 NaN 값을 바꾸어주면 되겠네요.

In [152]:
# 먼저 item_price가 결측치인것 부터 골라줄게요.
item_price_isnull = uriage_data['item_price'].isnull()
# item_price_isnull의 값이 True인 값 중에서 상품 명을 추출하고, 반복해줄게요.
for name in uriage_data.loc[item_price_isnull, 'item_name'].unique():
    price = uriage_data.loc[(~item_price_isnull)&(uriage_data['item_name']==name), 'item_price'].max()
    uriage_data.loc[(item_price_isnull)&(uriage_data['item_name']==name), 'item_price'] = price
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,1900.0,김우찬
2,2019-05-11 19:42,상품A,100.0,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품A,100.0,김강현


In [151]:
# 먼저 item_price가 결측치인것 부터 골라줄게요.
item_price_isnull = uriage_data['item_price'].isnull()
# item_price_isnull의 값이 True인 값 중에서 상품 명을 추출하고, 반복해줄게요.
for name in uriage_data[item_price_isnull]['item_name'].unique():
    price = uriage_data[(~item_price_isnull)&(uriage_data['item_name']==name)]['item_price'].max()
    uriage_data[(item_price_isnull)&(uriage_data['item_name']==name)]['item_price'] = price
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,NaN,김우찬
2,2019-05-11 19:42,상품A,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품A,NaN,김강현


In [109]:
# 자 혹시 첫번째 예시와 두번째 예시의 차이를 아시나요?
# 첫번째 예시에서는 loc를 사용하지 않고, 값에 접근을 했는데 첫번째 예시의 경우에 
# 인덱싱 표현은 먼저 uriage_data[item_price_isnull]로 결측치가 있는 행만 필터링한 후, 그 결과에서 'item_name' 열을 선택하는 것입니다.
# 이 과정에서 반환되는 객체는 원래 데이터프레임의 일부이고, 결과는 그 행들의 복사본이 될 수 있습니다. 
# 따라서 원래 데이터프레임에 영향을 주지 않는 방식입니다.
# loc 표현은 .loc 인덱서를 사용하여, item_price_isnull 조건을 만족하는 행과 'item_name' 열을 동시에 선택합니다.
# .loc를 사용하면 결과는 원본 데이터프레임의 특정 위치를 참조하기 때문에, 값의 할당이나 변경이 원래 데이터프레임에 영향을 미칩니다.

In [153]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,1900.0,김우찬
2,2019-05-11 19:42,상품A,100.0,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품A,100.0,김강현


In [155]:
# 결측치가 하나도 없네요. 만세~
uriage_data.isnull().sum()

purchase_date    0
item_name        0
item_price       0
customer_name    0
dtype: int64

In [164]:
# 이제 customer_name을 기준으로 두 데이터를 하나로 합쳐보죠.
new_data = pd.merge(left=kokyaku_data,
                   right=uriage_data,
                   left_on='고객이름',
                   right_on='customer_name',
                   how='left')
new_data.head()

,고객이름,지역,등록일,purchase_date,item_name,item_price,customer_name
0,김현성,H시,2018-01-04,2019-02-24 1:07,상품C,300.0,김현성
1,김현성,H시,2018-01-04,2019-05-08 15:42,상품P,1600.0,김현성
2,김현성,H시,2018-01-04,2019-07-03 7:49,상품M,1300.0,김현성
3,김현성,H시,2018-01-04,2019-01-02 13:52,상품L,1200.0,김현성
4,김현성,H시,2018-01-04,2019-06-29 4:58,상품R,1800.0,김현성


In [165]:
# 이름이 중복되니, 이름 값을 제거해줄게요.
new_data = new_data.drop('customer_name', axis=1)
new_data.head()
# item_name을 기준으로 정렬시켜보죠.
new_data = new_data.sort_values('item_name', ignore_index=True)

In [166]:
new_data.head()

,고객이름,지역,등록일,purchase_date,item_name,item_price
0,김승훈,C시,2017-01-30,2019-06-21 11:09,상품A,100.0
1,김주안,G시,2017-05-05,2019-05-16 2:34,상품A,100.0
2,김하진,C시,2018-05-18,2019-01-15 22:59,상품A,100.0
3,김지호,D시,2017-06-16,2019-05-02 1:45,상품A,100.0
4,김수호,C시,2018-04-14,2019-05-07 14:29,상품A,100.0


In [167]:
# 년과 월데이터만 따로 추가해보죠.
new_data['년/월'] = pd.to_datetime(new_data['등록일'])
new_data['년/월'] = new_data['년/월'].dt.strftime('%Y%m')
new_data.head()

,고객이름,지역,등록일,purchase_date,item_name,item_price,년/월
0,김승훈,C시,2017-01-30,2019-06-21 11:09,상품A,100.0,201701
1,김주안,G시,2017-05-05,2019-05-16 2:34,상품A,100.0,201705
2,김하진,C시,2018-05-18,2019-01-15 22:59,상품A,100.0,201805
3,김지호,D시,2017-06-16,2019-05-02 1:45,상품A,100.0,201706
4,김수호,C시,2018-04-14,2019-05-07 14:29,상품A,100.0,201804


In [173]:
# 자 이제 어떤 고객이, 어떤 상품을 샀는지 파악해볼까요?
new_data.pivot_table(index='고객이름',
                    columns='item_name',
                    aggfunc='size',
                    fill_value=0)

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
고객이름,,,,,,,,,,,,,,,,,,,,,
김가온,3,0,2,0,0,0,0,0,1,1,...,0,0,0,2,2,0,1,2,0,0
김강민,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
김강현,6,0,1,2,1,0,0,1,1,0,...,0,0,2,0,1,0,1,0,0,0
김건우,0,0,0,0,3,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
김건희,2,1,0,0,1,0,1,0,0,0,...,0,1,0,0,1,4,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
김호준,0,0,0,0,0,0,0,0,0,1,...,0,0,2,0,0,1,1,0,1,0
정도형,1,0,1,1,0,0,0,0,0,0,...,0,2,3,0,0,0,1,0,2,0
정영훈,1,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,2,0
